In [1]:
import torch
import lovely_tensors as lt
import wandb
import tqdm
from models import SupervenientFeatureNetwork, CLUB, DecoupledSmileMIEstimator, DownwardSmileMIEstimator
lt.monkey_patch()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.empty_cache()

print(device)

cpu


In [2]:

# config =  {'batch_size': 1000, 'feature_size': 8, 'clip': 1, 'critic_output_size': 20, 'downward_hidden_sizes_v_critic': [192,192,192,192], 'downward_hidden_sizes_xi_critic': [164,164,164,164], 'feature_hidden_sizes': [576,576,576,576], 'decoupled_critic_hidden_sizes': [113,113,113,113], 'feature_lr': 0.0008501232325785077, 'decoupled_critic_lr': 0.00016297738002821739, 'downward_lr': 7.91160518491613e-05, 'update_f_every_N_steps': 1, 'weight_decay': 1.4496890117592044e-05, 'bias': True, 'num_atoms': 64}

# Train a feature network for a given config

In [3]:
def train_feature_network(config):

    dataset = torch.load("data/ecog_data_pairs.pth")
    trainloader = torch.utils.data.DataLoader(dataset, batch_size=config['batch_size'], shuffle=True)

    print('starrting')

    wandb.init(project="getting-figures", config=config)

    feature_network = SupervenientFeatureNetwork(
        num_atoms=config['num_atoms'],
        feature_size=config['feature_size'],
        hidden_sizes=config['feature_hidden_sizes'],
        include_bias=config['bias']
        ).to(device)
    decoupled_MI_estimator = DecoupledSmileMIEstimator(
        feature_size=config['feature_size'],
        critic_output_size=config['critic_output_size'],
        hidden_sizes=config['decoupled_critic_hidden_sizes'],
        clip=config['clip'],
        include_bias=config['bias']
        ).to(device)
    downward_MI_estimators = [
        DownwardSmileMIEstimator(
            feature_size=config['feature_size'],
            critic_output_size=config['critic_output_size'],
            hidden_sizes_v_critic=config['downward_hidden_sizes_v_critic'],
            hidden_sizes_xi_critic=config['downward_hidden_sizes_xi_critic'],
            clip=config['clip'],
            include_bias=config['bias']
            ).to(device) 
        for _ in range(config['num_atoms'])
    ]



    feature_optimizer = torch.optim.Adam(
        feature_network.parameters(),
        lr=config["feature_lr"],
        weight_decay=config["weight_decay"]
    )
    decoupled_optimizer = torch.optim.Adam(
        decoupled_MI_estimator.parameters(),
        lr=config["decoupled_critic_lr"],
        weight_decay=config["weight_decay"]
    )
    downward_optims = [
        torch.optim.Adam(
            dc.parameters(),
            lr=config["downward_lr"],
            weight_decay=config["weight_decay"]
        ) 
        for dc in downward_MI_estimators
    ]


    # TODO: figure out why only f network is being watched, I would like to keep a closer eye on the grad n params.
    # TODO: Look at how GANs are trained with pytorch and make sure I'm not doing anything unreasonable.
    # Eg, https://github.com/eriklindernoren/PyTorch-GAN/blob/master/implementations/gan/gan.py 
    # ^ this does not require retain_graph=True, so maybe this can be optomized somehow
    wandb.watch(feature_network, log='all')
    wandb.watch(decoupled_MI_estimator, log="all")
    for dc in downward_MI_estimators:
        wandb.watch(dc, log='all')

    ##
    ## TRAIN FEATURE NETWORK
    ##

    epochs = 3

    for _ in tqdm.tqdm(range(epochs), desc='Training'):
        for batch_num, batch in enumerate(trainloader):
            x0 = batch[:, 0].to(device).float()
            x1 = batch[:, 1].to(device).float()
            v0 = feature_network(x0)
            v1 = feature_network(x1) 

            # update decoupled critic
            feature_network.eval()
            decoupled_MI_estimator.train()
            downward_MI_estimators = [dc.train() for dc in downward_MI_estimators]

            decoupled_optimizer.zero_grad()
            decoupled_MI = decoupled_MI_estimator(v0, v1)
            decoupled_loss = -decoupled_MI
            decoupled_loss.backward(retain_graph=True)
            decoupled_optimizer.step()

            # update each downward critic 
            for i in range(config['num_atoms']):
                downward_optims[i].zero_grad()
                channel_i = x0[:, i].unsqueeze(1)
                downward_MI_i = downward_MI_estimators[i](v1, channel_i)
                downward_loss = - downward_MI_i
                downward_loss.backward(retain_graph=True)
                downward_optims[i].step()
                # wandb.log({
                #     f"downward_MI_{i}": downward_MI_i   
                # })

            # update feature network   
            feature_network.train()
            decoupled_MI_estimator.eval()
            downward_MI_estimators = [dc.eval() for dc in downward_MI_estimators]

            feature_optimizer.zero_grad()
            sum_downward_MI = 0

            # MIs = []
            for i in range(config['num_atoms']):
                channel_i = x0[:, i].unsqueeze(1)
                channel_i_MI = downward_MI_estimators[i](v1, channel_i)
                sum_downward_MI += channel_i_MI
                # MIs.append(channel_i_MI)

            decoupled_MI1 = decoupled_MI_estimator(v0, v1)

            #clipped_min_MIs = max(0, min(MIs))

            Psi = decoupled_MI1 - sum_downward_MI #+ (config['num_atoms'] - 1) * clipped_min_MIs
            feature_loss = -Psi

            if batch_num % config['update_f_every_N_steps'] == 0:
                feature_loss.backward()
                feature_optimizer.step()

            wandb.log({
                "decoupled_MI": decoupled_MI1,
                "sum_downward_MI": sum_downward_MI,
                "Psi": Psi,
            })

    # free the memory associated with anything that is not the feature network
    for dc in downward_MI_estimators:
        del dc
    del decoupled_MI_estimator
    del decoupled_optimizer
    del downward_optims
    del trainloader
    del dataset
        
    return feature_network



In [2]:
config = {
    "batch_size": 1000,
    "num_atoms": 64,
    "feature_size": 3,
    "clip": 5,
    "critic_output_size": 32,
    "downward_hidden_sizes_v_critic": [512, 512, 512],
    "downward_hidden_sizes_xi_critic": [512, 512, 512],
    "feature_hidden_sizes": [256, 256, 256],
    "decoupled_critic_hidden_sizes": [512, 512, 512],
    "feature_lr": 1e-5,
    "decoupled_critic_lr": 1e-4,
    "downward_lr": 1e-4,    
    "bias": True,
    "update_f_every_N_steps": 5,
    "weight_decay": 1e-4,
}


In [4]:



feature_network = train_feature_network(config)



starrting


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: dmcsharry. Use `wandb login --relogin` to force relogin


Training: 100%|██████████| 3/3 [07:05<00:00, 141.98s/it]


# Finding the MI between different channels

I( xt_i ; xt_j)

In [ ]:

dataset = torch.load("data/ecog_data_pairs.pth")
trainloader = torch.utils.data.DataLoader(dataset, batch_size=2000, shuffle=False)

wandb.init(project="Interchannel MI")

channel_MI_estimator = DownwardSmileMIEstimator(
    feature_size=1, # replacing the feature with a channel which is dim 1
    critic_output_size=8,
    hidden_sizes_v_critic=[64, 512, 1028, 512],
    hidden_sizes_xi_critic=[64, 512, 1028, 512],
    clip=5,
    include_bias=True
).to(device) 

channel_MI_optim = torch.optim.Adam(channel_MI_estimator.parameters(), lr=1e-5, weight_decay=1e-6)

epochs = 20

channels = (1,2)

for _ in tqdm.tqdm(range(epochs), desc='Training a SMILE estimator for interchannel MI'):
    for batch_num, batch in enumerate(trainloader):
        x0 = batch[:, 0].to(device).float()
        x1 = batch[:, 1].to(device).float()

        channel_i = x0[:, channels[0]].unsqueeze(1)
        channel_j = x0[:, channels[1]].unsqueeze(1)

        MI = channel_MI_estimator(channel_j, channel_i)
        loss = -MI 
        loss.backward()
        channel_MI_optim.step()
        wandb.log({
            "Inter-channel MI": MI
        })


# Estimating Psi given a frozen feature network

In [3]:

# model_path = "/vol/bitbucket/dm2223/info-theory-experiments/promising_hmmm_f.pth"

# feature_network = SupervenientFeatureNetwork(
#     num_atoms=config['num_atoms'],
#     feature_size=config['feature_size'],
#     hidden_sizes=config['feature_hidden_sizes'],
#     include_bias=config['bias']
#     ).to(device)
# feature_network.load_state_dict(torch.load(model_path))
# feature_network.eval()



def find_true_Psi(feature_network, run_id, feature_config):

    config = {
        "batch_size": 600,
        "num_atoms": 64,
        "feature_size": feature_config['feature_size'],
        "clip": 5,
        "critic_output_size": 16,
        "downward_hidden_sizes_v_critic": [1028, 1028, 512, 64],
        "downward_hidden_sizes_xi_critic": [512, 512, 512, 64],
        "feature_hidden_sizes": [1028, 1028, 256],
        "decoupled_critis_hidden_sizes": [512, 512, 128],
        "decoupled_critic_lr": 1e-4,
        "downward_lr": 1e-4,
        "bias": True,
        "weight_decay": 1e-6,
        "original_run_id": run_id
    }

    dataset = torch.load("data/ecog_data_pairs.pth")
    trainloader = torch.utils.data.DataLoader(dataset, batch_size=config['batch_size'], shuffle=True)

    wandb.init(project="Finding-true-Psi-for-f", config=config, id=run_id)

    decoupled_critic = DecoupledSmileMIEstimator(
        feature_size=config['feature_size'],
        critic_output_size=config['critic_output_size'],
        hidden_sizes=config['decoupled_critis_hidden_sizes'],
        clip=config['clip'],
        include_bias=config['bias']
        ).to(device)

    downward_critics = [
        DownwardSmileMIEstimator(
            feature_size=config['feature_size'],
            critic_output_size=config['critic_output_size'],
            hidden_sizes_v_critic=config['downward_hidden_sizes_v_critic'],
            hidden_sizes_xi_critic=config['downward_hidden_sizes_xi_critic'],
            clip=config['clip'],
            include_bias=config['bias']
            ).to(device) 
        for _ in range(config['num_atoms'])
    ]

    downward_optims = [
        torch.optim.Adam(
            dc.parameters(),
            lr=config["downward_lr"],
            weight_decay=config["weight_decay"]
        ) 
        for dc in downward_critics
    ]

    decoupled_optimizer = torch.optim.Adam(
        decoupled_critic.parameters(),
        lr=config["decoupled_critic_lr"],
        weight_decay=config["weight_decay"]
    )

    # TODO: figure out why only f network is being watched, I would like to keep a closer eye on the grad n params.
    # TODO: Look at how GANs are trained with pytorch and make sure I'm not doing anything unreasonable.
    # Eg, https://github.com/eriklindernoren/PyTorch-GAN/blob/master/implementations/gan/gan.py 
    # ^ this does not require retain_graph=True, so maybe this can be optomized somehow
    wandb.watch(decoupled_critic, log="all")
    for dc in downward_critics:
        wandb.watch(dc, log='all')

    epochs = 5

    for _ in tqdm.tqdm(range(epochs), desc='Training'):
        for _, batch in enumerate(trainloader):
            x0 = batch[:, 0].to(device).float()
            x1 = batch[:, 1].to(device).float()

            # update decoupled critic

            v0 = feature_network(x0)
            v1 = feature_network(x1) 

            decoupled_optimizer.zero_grad()
            decoupled_MI = decoupled_critic(v0, v1)
            decoupled_loss = -decoupled_MI
            decoupled_loss.backward(retain_graph=True)
            decoupled_optimizer.step()


            # update each downward critic 

            MIs = []

            for i in range(config['num_atoms']):
                downward_optims[i].zero_grad()
                channel_i = x0[:, i].unsqueeze(1)
                downward_MI_i = downward_critics[i](v1, channel_i)
                # add spectral norm to the loss
                downward_loss = - downward_MI_i
                downward_loss.backward(retain_graph=True)
                downward_optims[i].step()
                wandb.log({
                    f"downward_MI_{i}": downward_MI_i   
                })
                MIs.append(downward_MI_i)

            # update feature network   

            min_MI = min(MIs)
            clipped_min_MIs = max(0, min_MI)

            sum_downward_MI = 0

            for i in range(config['num_atoms']):
                channel_i = x0[:, i].unsqueeze(1)
                sum_downward_MI += downward_critics[i](v1, channel_i)

            decoupled_MI1 = decoupled_critic(v0, v1)

            Psi = decoupled_MI1 - sum_downward_MI + (config['num_atoms'] - 1) * clipped_min_MIs

            wandb.log({
                "decoupled_MI": decoupled_MI1,
                "sum_downward_MI": sum_downward_MI,
                "Psi": Psi,
            })
        
    return Psi



In [6]:
# save feature network
torch.save(feature_network.state_dict(), f"/vol/bitbucket/dm2223/info-theory-experiments/another_model.pth")

In [6]:
# load feature network
feature_network = SupervenientFeatureNetwork(
    num_atoms=config['num_atoms'],
    feature_size=config['feature_size'],
    hidden_sizes=config['feature_hidden_sizes'],
    include_bias=config['bias']
    ).to(device)
# feature_network.load_state_dict(torch.load("/vol/bitbucket/dm2223/info-theory-experiments/latest_model.pth"))
feature_network.load_state_dict(torch.load("/Users/davidmcsharry/dev/imperial/info-theory-experiments/emergent_feature_network_omg.pth", map_location=torch.device('cpu') ))
# find true Psi
Psi = find_true_Psi(feature_network, run_id="emerg", feature_config=config)

FileNotFoundError: [Errno 2] No such file or directory: 'data/ecog_data_pairs.pth'

In [17]:

import optuna
def objective(trial):

    config = {
        "batch_size": trial.suggest_categorical("batch_size", [500, 1000, 2000]),
        "num_atoms": 64,
        "feature_size": trial.suggest_categorical("feature_size", [2, 4, 8, 16]),
        "clip": trial.suggest_int("clip", 1, 10),
        "critic_output_size": trial.suggest_int("critic_output_size", 8, 64, log=True),
        "downward_hidden_sizes_v_critic": [trial.suggest_int("downward_hidden_size_v", 64, 512, log=True) for _ in range(3)],
        "downward_hidden_sizes_xi_critic": [trial.suggest_int("downward_hidden_size_xi", 32, 256, log=True) for _ in range(3)],
        "feature_hidden_sizes": [trial.suggest_int("feature_hidden_size", 256, 1024, log=True) for _ in range(4)],
        "decoupled_critis_hidden_sizes": [trial.suggest_int("decoupled_critic_hidden_size", 64, 512, log=True) for _ in range(3)],
        "feature_lr": trial.suggest_float("feature_lr", 1e-6, 1e-3, log=True),
        "decoupled_critic_lr": trial.suggest_float("decoupled_critic_lr", 1e-5, 1e-3, log=True),
        "downward_lr": trial.suggest_float("downward_lr", 1e-5, 1e-3, log=True),
        "bias": True,
        "update_f_every_N_steps": trial.suggest_int("update_f_every_N_steps", 1, 20, log=True),
        "weight_decay": trial.suggest_float("weight_decay", 1e-9, 1e-4, log=True),
    }

    feature_network = train_feature_network(config)
    Psi = find_true_Psi(feature_network, wandb.run.id, feature_config=config)

    return Psi

study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=20)  # Adjust the number of trials as needed

# add the Psi value the params of the top 5 runs, then save

import json

# save best params
with open("optuna_results/best_params.json", "w") as f:
    json.dump(study.best_params, f)

# save second best params
with open("optuna_results/second_best_params.json", "w") as f:
    json.dump(study.best_trials[1].params, f)

# save third best params
with open("optuna_results/third_best_params.json", "w") as f:
    json.dump(study.best_trials[2].params, f)

# save fourth best params
with open("optuna_results/fourth_best_params.json", "w") as f:
    json.dump(study.best_trials[3].params, f)

# save fifth best params
with open("optuna_results/fifth_best_params.json", "w") as f:
    json.dump(study.best_trials[4].params, f)


top_5 = study.best_trials[:5]

for i, trial in enumerate(top_5):
    trial.params['Psi'] = trial.value
    with open(f"optuna_results/trial_{i}.json", "w") as f:
        json.dump(trial.params, f)


[I 2024-04-15 02:40:40,595] A new study created in memory with name: no-name-09a86fe3-ffd4-45df-8ceb-77f5b43d8ebc


Psi,▁
decoupled_MI,▁
downward_MI_0,▁
downward_MI_1,▁
downward_MI_10,▁
downward_MI_11,▁
downward_MI_12,▁
downward_MI_13,▁
downward_MI_14,▁
downward_MI_15,▁
downward_MI_16,▁


Training:   0%|          | 0/1 [00:00<?, ?it/s]


<class '__main__.SupervenientFeatureNetwork'>


Psi,▁
decoupled_MI,▁
downward_MI_0,▁
downward_MI_1,▁
downward_MI_10,▁
downward_MI_11,▁
downward_MI_12,▁
downward_MI_13,▁
downward_MI_14,▁
downward_MI_15,▁
downward_MI_16,▁


...

Psi,▁
decoupled_MI,▁
downward_MI_0,▁
downward_MI_1,▁
downward_MI_10,▁
downward_MI_11,▁
downward_MI_12,▁
downward_MI_13,▁
downward_MI_14,▁
downward_MI_15,▁
downward_MI_16,▁


Training:   0%|          | 0/1 [00:00<?, ?it/s]


<class '__main__.SupervenientFeatureNetwork'>


Psi,▁
decoupled_MI,▁
downward_MI_0,▁
downward_MI_1,▁
downward_MI_10,▁
downward_MI_11,▁
downward_MI_12,▁
downward_MI_13,▁
downward_MI_14,▁
downward_MI_15,▁
downward_MI_16,▁


Training:   0%|          | 0/1 [00:00<?, ?it/s]
[I 2024-04-15 02:41:08,486] Trial 1 finished with value: -0.0007123947143554688 and parameters: {'batch_size': 2000, 'feature_size': 16, 'clip': 2, 'critic_output_size': 10, 'downward_hidden_size_v': 476, 'downward_hidden_size_xi': 75, 'feature_hidden_size': 958, 'decoupled_critic_hidden_size': 175, 'feature_lr': 1.723749852560971e-05, 'decoupled_critic_lr': 1.6145668368018935e-05, 'downward_lr': 6.425805952319134e-05, 'update_f_every_N_steps': 5, 'weight_decay': 1.5989586833270346e-06}. Best is trial 1 with value: -0.0007123947143554688.


# Hacky hyperparam search